In [1]:
import pandas as pd 
import os
import pickle
def savepickle(filename,objecttopickle):
    with open(filename,'wb') as handle:
        pickle.dump(objecttopickle,handle,protocol=pickle.HIGHEST_PROTOCOL)
        
def openpickle(filename):
    with open(filename, 'rb') as handle:
        newname = pickle.load(handle)
    return newname
 
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys

In [45]:
#Ran on macbook pro os Catalina 10.15.4
#python and package versions
#Chrome driver Version 83.0.4103.61
from platform import python_version

print('Python (Anaconda build)', python_version())
print('Pandas: ',pd.__version__)
print('Selenium: ',webdriver.__version__)

Python (Anaconda build) 3.7.3
Pandas:  1.0.3
Selenium:  3.14.1


In [2]:
#Load pickle with all the presentation info
pinfo=openpickle('presinfo1419.pkl')


In [3]:
#Extract the presentation numbers
pids=[]
for k,v in pinfo.items():
    for key,val in v.items():
        pids.append(key)
pids=[x.split('_') for x in pids]
pids=[[x[0][-2:],x[1]] for x in pids]

In [4]:
#Create a dictionary with all the urls for each presentations, keyed by presentation ID
ourl1='https://convention2.allacademic.com/one/asc/asc'
ourl2='/index.php?cmd=Online+Program+View+Paper&selected_paper_id='
purls={x:{} for x in range(2014,2020)}
for x in pids:
    purls[int('20'+x[0])]['20'+x[0]+'_'+x[1]]=ourl1+x[0]+ourl2+x[1]


In [5]:
#Creates a dict where the urls are associated with a numeric id I can use to identify where the code breaks
#and the reverse to reassign the abstract ids once its done
fnumdct={}
purls2={}
cnt=0
for k,v in purls.items():
    for f,u in v.items():
        cnt+=1
        purls2[cnt]=u
        fnumdct[cnt]=f
savepickle('purls2.pkl',purls2)  
savepickle('fnumdct.pkl',fnumdct)

        

In [ ]:
purls2=openpickle('purls2.pkl')  
fnumdct=openpickle('fnumdct.pkl')

In [6]:
#Headless driver--need to download chrome driver and put in same path (or adjust path)--get same as your Chrome version
#https://chromedriver.storage.googleapis.com/index.html?path=83.0.4103.39/
#For some reason, I run into an OS error (OSError: [Errno 24] Too many open files) 
#that I haven't been able to fix. Workaround is to let the code run until it fails, restart kernel 
#and start again where it left off. 

abst={}
cnt=14188
chrome_options = Options()  
chrome_options.add_argument("--headless")
for k,v in purls2.items():
    if k>cnt:

   

        driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"),options=chrome_options)

        driver.get(v)
        el=driver.find_elements_by_tag_name('blockquote')
        try:
            abst[k]=el[0].get_attribute('innerHTML')
            driver.quit()
        except:
            abst[k]='MISSING'
            driver.quit()
            
            
            
    
    

In [7]:
#Get the last abstract that was scrapped before error
#Restart kernel and replace the value of cnt before running the scrapper loop
cnt=max(list(abst.keys()))
print(cnt)

In [9]:
#Reassign the abstract ids before saving
abst2={}
for k,v in abst.items():
    abst2[fnumdct[k]]=v


In [10]:
savepickle('abstracts/abstracts_part3.pkl',abst2) 

In [9]:
#Old version. Works but opens instance of Google Chrome for each url. Annoying but working.
#TODO:figure out how to make driver headless
abst={}
cnt=0

for yr,v in purls.items():

    for k,p in v.items():
        #print(k)
        driver = webdriver.Chrome()#executable_path=pJSpath
        driver.get(p)
        el=driver.find_elements_by_tag_name('blockquote')
        try:
            abst[k]=el[0].get_attribute('innerHTML')
        except:
            abst[k]='MISSING'
        driver.close()
        
    
savepickle('abstracts/abstracts1419.pkl',abst)    

OSError: [Errno 24] Too many open files

In [14]:
#Load the seperate abstract dict
a1=openpickle('abstracts/abstracts_part1.pkl')
a2=openpickle('abstracts/abstracts_part2.pkl')
a3=openpickle('abstracts/abstracts_part3.pkl')

In [19]:
#Merge into one dict
allabst={}
allabst.update(a1)
allabst.update(a2)
allabst.update(a3)

In [35]:
savepickle('allabstracts1419.pkl',allabst)

In [27]:
#save abstracts as csv
df2=pd.DataFrame.from_dict(allabst, orient='index')
#df2=df2[df2[0]!='MISSING']
df2.to_csv('asc_abstracts1419.csv')